### Posibilidad de identificación de un sujeto despues de la anonimización de los datos

Para que un dato sea realmente anónimo, su identificación y su relación con un sujeto ha de ser completamente irreversible. Es decir,  podemos decir que un dato es anónimo si la posibilidad de vinculación con la persona a la que hubiera identificado el dato es nula teniendo en cuenta que se pueden utilizar otros datos y recursos y que estos pueden aumentar el riesgo de identificación con el tiempo.

Actualmente la anonimizacion hecha por el departamento de gestión académica que nos proporció los datos consiste en cambiar el niub de los alumnos por un identificador único que no tiene ninguna relación alfa-numérica con el niub. Es decir, sabiendo solo los identificadores de los alumnos es imposible saber de quien se trata realmente ya que  no se sabe la manera ni el orden de asignar los nuevos identificadores.
Debido a que prácticamente no se ha aplicado ninguna técnica de anonimizacion (a parte del remplazo del niub por un identificador aleatorio), si se dispone de varios datos sobre un sujeto en particular, se podría llegar a identificarlo.

Por ejemplo, se exponen los siguientes casos hipotéticos que podrían llevar a la identificación de un sujeto


In [1]:
import numpy as np
import pandas as pd

In [72]:
def select_rows(df, group, col):
    out = pd.DataFrame()
    tmp = df.copy()
    for s in group:
        out = pd.concat([out, tmp[tmp[col] == s]])
    out = out.reset_index(drop=True)
    return out

In [73]:
assig = pd.read_csv("datos2/assigs.csv", na_values=np.nan)
conv = pd.read_csv("datos2/conv.csv", na_values=np.nan)
registers = pd.read_csv("datos2/registers.csv", na_values=np.nan)
qualifications = pd.read_csv("datos2/qualifications.csv", na_values=np.nan)
datos = pd.merge(registers, qualifications)
print datos.columns

Index([u'id_alumne', u'sexe', u'naixement', u'nacionalitat', u'simultaneitat', u'becat', u'priv_pub_cfgs', u'tipus_lloc_cfgs', u'lloc_cfgs', u'priv_pub_secundaria', u'tipus_lloc_secundaria', u'lloc_secundaria', u'desc_via_acces', u'id_via_acces', u'any_pau', u'universitat_procedencia', u'universitat_procedencia_desc', u'sistema_educatiu_estranger', u'pais_sistema_estranger', u'nota_acces', u'desc_enseny', u'id_enseny', u'any_primera_matricula', u'any_matriculacio_assig', u'id_assig', u'tipus_apunt', u'nota_primera_conv', u'nota_segona_conv'], dtype='object')


#####A continuación intentaremos a partir de unos casos ipoteticos la identificación de una o varias personas
###Ejemplo 1. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Año de nacimiento: 1986
+ Sexo: hombre
+ Población del centro de secundaria: Castelldefels (comarca Baix Llobregat)

A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Primero filtramos los registros por el año de nacimeinto de las personas: 1986

In [74]:
datos1 = datos[datos['naixement'] == 1986].reset_index(drop=True)
print "Numero de alumnos nacido el 1986: " + str(len(datos.id_alumne.unique()))

Numero de alumnos nacido el 1986: 803


#####Podremos ver que la lista de alumnos se ha reducido a 22.

A continuación filtraremos los registros segun el sexo de tal manera que nos quedemos solo con los varones.


In [75]:
datos1 = datos1[datos1['sexe'] == 'H'].reset_index(drop=True)
print "Numero de alumnos varones que han nacido el 1986: " + str(len(datos.id_alumne.unique()))

Numero de alumnos varones que han nacido el 1986: 803


#####Podremos ver que la lista de alumnos se ha reducido a 21.

A continuación filtraremos los registros segun la comarca donde han cursado la secundario, en este caso Baix Llobregat

In [76]:
datos1 = datos1[datos1['lloc_secundaria'] == 'Baix Llobregat'].reset_index(drop=True)
print "Numero de alumnos varones que han nacido el 1986 y que han cursado la secundaria en la comarca de Baix Llobregat: " + str(len(datos1.id_alumne.unique()))

Numero de alumnos varones que han nacido el 1986 y que han cursado la secundaria en la comarca de Baix Llobregat: 1


##### Como podremos ver con estos tres datos que sabiamos sobre una persona pudimos identificarla dentro de la base de datos de este modo pudiendo ver toda su información.

#### Una posible solución al problema de este caso sería añadir un pequeño ruido a los datos que representan los años o subir el nivel que de la area administrativa a una provincia en vez de una comarca.

###Ejemplo 2. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Nota de las pruebas PAU: 6.915


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la nota de las Pruebas PAU: 6.915

In [77]:
datos2 = datos[datos['nota_acces'] == 6.915].reset_index(drop=True)
print "Numero de alumnos con la nota de las pruebas PAU igual a 6.915: " + str(len(datos2.id_alumne.unique()))

Numero de alumnos con la nota de las pruebas PAU igual a 6.915: 1


##### Como podremos ver solo con saber la nota de las pruebas PAU de una persona, pudimos identificarla dentro de la base de datos

#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales

###Ejemplo 3. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Notas de las asignaturas cursadas: 5.8, 8.9, 5.5, 6.2, 7.8


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la lista de notas mas arriba indicados.

In [90]:
notas = [5.8, 8.9, 5.5, 6.2, 7.8]

uno = datos[datos['nota_primera_conv'] == 5.8].reset_index(drop=True)
dos = datos[datos['nota_primera_conv'] == 8.9].reset_index(drop=True)
tres = datos[datos['nota_primera_conv'] == 5.5].reset_index(drop=True)
cuatro = datos[datos['nota_primera_conv'] == 6.2].reset_index(drop=True)
cinco = datos[datos['nota_primera_conv'] == 7.8].reset_index(drop=True)

datos3 = pd.concat([uno, dos, tres, cuatro, cinco])
ids = datos3.id_alumne.unique()
gg = datos3.groupby('id_alumne')

id_alumnes = []

for id_alumne in ids:
    alumne = gg.get_group(id_alumne)['nota_primera_conv'].values
    if (len(alumne) >= len(notas)):
        k = 0
        for nota in notas:
            if nota in alumne:
                k+=1
        if k == len (notas):
            id_alumnes.append(gg.get_group(id_alumne)['id_alumne'].unique())
            

print "Numero de alumnos encontrados con las siguientes notas " + str(notas) + ": " + str(len(id_alumnes))

Numero de alumnos encontrados con las siguientes notas [5.8, 8.9, 5.5, 6.2, 7.8]: 1


#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales

###Ejemplo 4. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Nota de la asignatura cursada de Algebra (364291): 8.5
+ Nota de la asignatura cursada de : Electronica (364305): 7.7


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la lista de notas mas arriba indicados.

In [91]:
asignaturas = [364291, 364305]
datos4_A = datos[datos['id_assig'] == 364291].reset_index(drop=True)
datos4_B = datos[datos['id_assig'] == 364305].reset_index(drop=True)

datos4_A = datos4_A[datos4_A['nota_primera_conv'] == 8.5]
datos4_B = datos4_B[datos4_B['nota_primera_conv'] == 7.7]

In [94]:
datos4 = pd.concat([datos4_A, datos4_B])
ids = datos4.id_alumne.unique()
datos4 = datos4.groupby('id_alumne')

id_alumnes = []

for id_alumne in ids:
    gr = datos4.get_group(id_alumne)['id_assig'].values
    if (len(gr) >= len(asignaturas)):
        k = 0
        for asig in asignaturas:
            if asig in gr:
                k+=1
        if k == len (asignaturas):
            id_alumnes.append(datos4.get_group(id_alumne)['id_alumne'].unique())
        
print "Numero de alumnos encontrados: " + str(len(id_alumnes))


Numero de alumnos encontrados: 1


#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales